In [1]:
%load_ext autoreload
%autoreload 2
import pudl
from pudl import constants as pc
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [3]:
pudl_settings = pudl.workspace.setup.get_defaults()
eia923_tables = pc.pudl_tables['eia923']
eia923_years = [2018, 2019]
eia860_tables = pc.pudl_tables['eia860']
eia860_years = [2018, 2019]
sandbox = True

In [ ]:
ds = pudl.workspace.datastore.Datastore(
    Path(pudl_settings["pudl_in"]),
    sandbox=sandbox)
# Extract EIA forms 923, 860
eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)
eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)

# Transform EIA forms 923, 860
eia860_transformed_dfs = pudl.transform.eia860.transform(
    eia860_raw_dfs, eia860_tables=eia860_tables)
eia923_transformed_dfs = pudl.transform.eia923.transform(
    eia923_raw_dfs, eia923_tables=eia923_tables)

In [ ]:
# create an eia transformed dfs dictionary
eia_transformed_dfs = eia923_transformed_dfs.copy()
eia_transformed_dfs.update(eia860_transformed_dfs.copy())
# convert types..
eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(
    eia_transformed_dfs, 'eia')

entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
    eia_transformed_dfs,
    eia860_years=eia860_years,
    eia923_years=eia923_years,
)

In [ ]:
list(eia_transformed_dfs.keys())

In [ ]:
eia_transformed_dfs['generation_fuel_eia923'].sort_values('report_date', ascending=False)